Saco promedio diario mensual (promedio de cada dia cada mes)

Hago un  ranking de las radiaciones. Para saca ENFICC


3.Tengo dos series de datos y sale 1.


El calculo de la efectividad del método sería ideal hacerlo para diferenmtes años y difrentes periodos (3, 6 y 12 meses)


In [14]:
import os
import pandas as pd
import statistics as st

In [44]:
## Funciones

def ghi_mensual(serie_horaria):
    agnos = sorted(serie_horaria['AÑO'].unique())
    ghi_df = pd.DataFrame(columns=["AÑO", "MES", "DIAS", "GHImes"])
    for agno in agnos:
        for mes in range(1,13):
            nombre_energia = list(serie_horaria)[-1]
            dias = sorted(serie_horaria[(serie_horaria['AÑO'] == agno) &
                        (serie_horaria['MES'] == mes)]["DIA"].unique())
            ghi_mes = serie_horaria.loc[((serie_horaria["AÑO"] == agno)
                            & (serie_horaria["MES"] == mes)),
                                        nombre_energia].sum()/1000
            ghi_dict = {"AÑO":agno, "MES":mes, "DIAS":dias[-1],
                        "GHImes":ghi_mes}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
    return ghi_df

def ghi_dia(serie_ghi_mensual):
    agnos = sorted(serie_ghi_mensual['AÑO'].unique())
    ghi_df = pd.DataFrame(columns=["AÑO", "MES", "GHIdiario"])
    for agno in agnos:
        for mes in range(1,13):
            dias = serie_ghi_mensual.loc[((serie_ghi_mensual["AÑO"] == agno)
                            & (serie_ghi_mensual["MES"] == mes))
                                         ,"DIAS"].item()
            ghi_diario = serie_ghi_mensual.loc[(
                (serie_ghi_mensual["AÑO"] == agno)
                            & (serie_ghi_mensual["MES"] == mes)),
                "GHImes"].item()/dias
            ghi_dict = {"AÑO":agno, "MES":mes, "GHIdiario":ghi_diario}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
    return ghi_df

def extraer_datos(fuente, ciudad):
    base_dir = os.path.join( '.',os.path.join( '02_taller01','data')) 
    output_dir = os.path.join( '.','output')
    serie_csv = os.path.join(base_dir,ciudad,fuente+'.csv')
    serie_pd = pd.read_csv(serie_csv, sep = ';',
                encoding='latin-1')
    return serie_pd

def enficc(datos,seguridad):
    ghi_mensual_loc = ghi_mensual(datos)
    ghi_diario_loc = ghi_dia(ghi_mensual_loc)
    percentil = 1-seguridad
    men = seguridad*100
    enficc = ghi_diario_loc.quantile(q=percentil, 
                                 numeric_only=True)["GHIdiario"]
    print('La ENFICC ',men, '% es: ' , enficc)
    
def ajustar_datos(corta, NREL):
    """Corta en array y larga en df"""
    
    larga= NREL.NREL
    sigma_corta = st.stdev(corta)
    sigma_larga = st.stdev(larga)
    prom_corta = st.mean(corta)
    prom_larga = st.mean(larga)
    ajustada = (prom_corta-(sigma_corta/sigma_larga)*prom_larga)+(sigma_corta/sigma_larga)*larga
    d = {"AÑO":NREL.AÑO, "MES":NREL.MES, "DIA":NREL.DIA, "HORA":NREL.HORA,"IDEAM":ajustada}
    datos = pd.DataFrame(data=d)
    datos=datos[["AÑO","MES", "DIA", "HORA","IDEAM"]]
    
    
    return datos

In [33]:
IDEAM_Valledupar = extraer_datos('IDEAM','Valledupar')
NREL_Valledupar = extraer_datos('NREL','Valledupar')
NREL_Valledupar.head(5)

,AÑO,MES,DIA,HORA,NREL
0,2005,1,1,0,0.0
1,2005,1,1,1,0.0
2,2005,1,1,2,0.0
3,2005,1,1,3,0.0
4,2005,1,1,4,0.0


In [46]:
IDEAM = IDEAM_Valledupar.sort_values(['AÑO','MES','DIA','HORA'])
IDEAM = IDEAM[(IDEAM.AÑO>2005) & (IDEAM.AÑO<2007)]
IDEAM = IDEAM.IDEAM
NREL = NREL_Valledupar
ajst = ajustar_datos(IDEAM,NREL)
enficc(ajst,0.99)

La ENFICC  99.0 % es:  4.61584192774
